In [ ]:
# Cell 1: Setup & Environment Installation

# --- ติดตั้ง Libraries ---
!pip install timm opendatasets albumentations scikit-learn --quiet

# --- Import Libraries ที่จำเป็น ---
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.notebook import tqdm
import timm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import cv2
import matplotlib.pyplot as plt
import glob

print("Setup Complete. Libraries are ready.")


In [ ]:
# Cell 2: Configuration
class CFG:
    # --- Paths ---
    DATA_DIR = './state-farm-distracted-driver-detection/'
    TRAIN_DIR = os.path.join(DATA_DIR, 'imgs/train')
    TEST_DIR = os.path.join(DATA_DIR, 'imgs/test')
    
    # --- Model ---
    MODEL_NAME = 'efficientnet_b0' # โมเดลที่สมดุลและมีประสิทธิภาพ
    PRETRAINED = True
    
    # --- Training Parameters ---
    IMG_SIZE = 224
    BATCH_SIZE = 64 # อาจต้องลดลงถ้า GPU memory ไม่พอ
    EPOCHS = 5 # ชุดข้อมูลนี้ใหญ่มาก เริ่มจากน้อยๆ ก่อน
    LEARNING_RATE = 1e-4
    
    # --- Environment ---
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    NUM_WORKERS = 2

print(f"Configuration loaded. Using device: {CFG.DEVICE}")



In [ ]:
# Cell 3: Data Acquisition & Preparation
import opendatasets as od

dataset_url = 'https://www.kaggle.com/c/state-farm-distracted-driver-detection'
od.download(dataset_url)

# --- สร้าง DataFrame จากข้อมูล Training ---
image_paths = []
labels = []

# วนลูปในแต่ละโฟลเดอร์คลาส (c0, c1, ...)
for class_name in sorted(os.listdir(CFG.TRAIN_DIR)):
    class_dir = os.path.join(CFG.TRAIN_DIR, class_name)
    if os.path.isdir(class_dir):
        for img_name in os.listdir(class_dir):
            image_paths.append(os.path.join(class_dir, img_name))
            labels.append(class_name)

train_df = pd.DataFrame({'image_path': image_paths, 'label': labels})

# --- แปลง Label เป็นตัวเลข ---
# ไม่จำเป็นต้องใช้ LabelEncoder เพราะชื่อคลาสเรียงลำดับ c0-c9 อยู่แล้ว
# เราสามารถดึงเลขจาก string ได้เลย
train_df['label_encoded'] = train_df['label'].str.slice(start=1).astype(int)

CFG.NUM_CLASSES = len(train_df['label'].unique())

print(f"Data prepared. Found {len(train_df)} training images in {CFG.NUM_CLASSES} classes.")
display(train_df.head())

# --- แสดงตัวอย่างภาพ ---
plt.figure(figsize=(12, 8))
for i in range(CFG.NUM_CLASSES):
    plt.subplot(2, 5, i + 1)
    sample_img_path = train_df[train_df['label_encoded'] == i].iloc[0]['image_path']
    img = cv2.imread(sample_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.title(f"Class: c{i}")
    plt.axis('off')
plt.tight_layout()
plt.show()



In [ ]:
# Cell 4: Augmentations and Custom Dataset

train_transforms = A.Compose([
    A.Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

valid_transforms = A.Compose([
    A.Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

class DriverDataset(Dataset):
    def __init__(self, df, transforms=None, is_test=False):
        self.df = df
        self.transforms = transforms
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # สำหรับ Test set จะใช้ 'img' column, Train set ใช้ 'image_path'
        image_path = os.path.join(CFG.TEST_DIR, row['img']) if self.is_test else row['image_path']
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            image = self.transforms(image=image)['image']
        
        if self.is_test:
            return image
        else:
            label = torch.tensor(row['label_encoded'], dtype=torch.long)
            return image, label



In [ ]:
# Cell 5: Create DataLoaders

# แบ่งข้อมูลโดยรักษาสัดส่วนของแต่ละคลาส
train_df, val_df = train_test_split(
    train_df, 
    test_size=0.2, 
    random_state=42, 
    stratify=train_df['label']
)

# สร้าง Datasets
train_dataset = DriverDataset(train_df, transforms=train_transforms)
val_dataset = DriverDataset(val_df, transforms=valid_transforms)

# สร้าง DataLoaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=CFG.BATCH_SIZE, 
    shuffle=True, 
    num_workers=CFG.NUM_WORKERS
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=CFG.BATCH_SIZE, 
    shuffle=False, 
    num_workers=CFG.NUM_WORKERS
)

print(f"DataLoaders created. Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")



In [ ]:
# Cell 6: Define Model, Loss, and Optimizer

# สร้างโมเดลจาก timm
model = timm.create_model(
    CFG.MODEL_NAME,
    pretrained=CFG.PRETRAINED,
    num_classes=CFG.NUM_CLASSES
).to(CFG.DEVICE)

# กำหนด Loss Function และ Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.LEARNING_RATE)

print(f"Model '{CFG.MODEL_NAME}' created with {CFG.NUM_CLASSES} output classes.")



In [ ]:
# Cell 7: The Training Loop

best_val_acc = 0.0
BEST_MODEL_PATH = f"{CFG.MODEL_NAME}_best.pth"

for epoch in range(CFG.EPOCHS):
    print(f"\n--- Epoch {epoch+1}/{CFG.EPOCHS} ---")
    
    # --- Training Phase ---
    model.train()
    train_loss, train_correct = 0, 0
    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(CFG.DEVICE), labels.to(CFG.DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        train_correct += torch.sum(preds == labels.data)

    # --- Validation Phase ---
    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validating"):
            images, labels = images.to(CFG.DEVICE), labels.to(CFG.DEVICE)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels.data)
            
    # คำนวณค่าเฉลี่ย
    avg_train_loss = train_loss / len(train_loader.dataset)
    avg_train_acc = train_correct.double() / len(train_loader.dataset)
    
    avg_val_loss = val_loss / len(val_loader.dataset)
    avg_val_acc = val_correct.double() / len(val_loader.dataset)
    
    print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f} | Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}")

    # บันทึกโมเดลที่ดีที่สุด
    if avg_val_acc > best_val_acc:
        print(f"Validation accuracy improved from {best_val_acc:.4f} to {avg_val_acc:.4f}. Saving model...")
        best_val_acc = avg_val_acc
        torch.save(model.state_dict(), BEST_MODEL_PATH)

print("\nFinished Training!")



In [ ]:
# Cell 8: Inference and Submission File Generation

# --- เตรียม Test Set ---
# รายชื่อไฟล์ภาพใน test set
test_image_paths = glob.glob(os.path.join(CFG.TEST_DIR, '*.jpg'))
test_df = pd.DataFrame({'img': [os.path.basename(p) for p in test_image_paths]})

test_dataset = DriverDataset(test_df, transforms=valid_transforms, is_test=True)
test_loader = DataLoader(
    test_dataset, 
    batch_size=CFG.BATCH_SIZE, 
    shuffle=False, 
    num_workers=CFG.NUM_WORKERS
)

# --- โหลดโมเดลและทำนายผล ---
model.load_state_dict(torch.load(BEST_MODEL_PATH))
model.eval()

all_preds_probs = []
with torch.no_grad():
    for images in tqdm(test_loader, desc="Generating Predictions"):
        images = images.to(CFG.DEVICE)
        outputs = model(images)
        # ใช้ Softmax เพื่อแปลง output เป็นความน่าจะเป็น
        probs = nn.functional.softmax(outputs, dim=1)
        all_preds_probs.extend(probs.cpu().numpy())

# --- สร้าง DataFrame สำหรับ Submission ---
submission_df = pd.DataFrame(all_preds_probs, columns=[f'c{i}' for i in range(CFG.NUM_CLASSES)])
submission_df.insert(0, 'img', test_df['img'])

# ตรวจสอบว่าผลรวมความน่าจะเป็นในแต่ละแถวเท่ากับ 1
# print(submission_df.iloc[:, 1:].sum(axis=1)) 

submission_df.to_csv('submission.csv', index=False)

print("\nsubmission.csv created successfully!")
display(submission_df.head())

